### IMPORT REQUIRED PACKAGES

In [72]:
!pip install llama-index-embeddings-clip
!pip install llama-index-multi-modal-llms-anthropic
!pip install llama-index-vector-stores-qdrant
!pip install llama_index ftfy regex tqdm
!pip install torch torchvision
!pip install matplotlib scikit-image
!pip install -U qdrant_client
%pip install llama-index-retrievers-bm25
!pip install llama_index.embeddings.fastembed
!pip install llama_index.llms.anthropic
!pip install llama-index-embeddings-clip
!pip install git+https://github.com/openai/CLIP.git
!pip install llama_index ftfy regex tqdm
!pip install torch torchvision
!pip install matplotlib scikit-image

In [78]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## SETUP EMBEDDING MODEL


In [8]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import Settings
Settings.embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

### SET UP CLAUDE LLM

In [31]:
import os

from llama_index.llms.anthropic import Anthropic
Settings.llm = Anthropic(model="claude-3-opus-20240229")

In [13]:
import nest_asyncio
nest_asyncio.apply()

INITIALISE QDRANT VECTOR STORE TO TEXT COLLECTION

In [32]:
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
import qdrant_client
from llama_index.core.indices import MultiModalVectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex

client = qdrant_client.QdrantClient(
    # embed_model=embed_model,
    url="https://43983594-507a-494f-98b2-99584b8978d7.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="lAEif06DWEGrD1QMBxXegmlyVhjMqI4MzcQ4zYLi8QyaVd0pO7TeAw",

)
aclient = qdrant_client.AsyncQdrantClient(
    url="https://43983594-507a-494f-98b2-99584b8978d7.europe-west3-0.gcp.cloud.qdrant.io:6333",
    api_key="lAEif06DWEGrD1QMBxXegmlyVhjMqI4MzcQ4zYLi8QyaVd0pO7TeAw",
)

text_store = QdrantVectorStore(client=client, aclient=aclient, collection_name="text_collection")
image_store = QdrantVectorStore(client=client, aclient=aclient, collection_name="image_collection")

text_index = VectorStoreIndex.from_vector_store(
  vector_store=text_store
)

## vector retriever
vector_retriever = text_index.as_retriever()



## INITIALISE QDRANT VECTOR STORE TO IMAGE COLLECTION

In [69]:
from llama_index.core.indices import MultiModalVectorStoreIndex

text_store = QdrantVectorStore(client=client, collection_name="text_collection")
image_store = QdrantVectorStore(client=client, collection_name="image_collection")

storage_context = StorageContext.from_defaults(
    vector_store=text_store, image_store=image_store,
)

image_index = MultiModalVectorStoreIndex.from_documents(

    [],
    storage_context=storage_context,
    show_progress = True
)



Parsing nodes: 0it [00:00, ?it/s]

Multimodal index built successfully!


### INITIALISE QUERY ENGINE WHICH PERFORMS RAG FUSION

In [95]:
from llama_index.core.prompts import PromptTemplate
from llama_index.core.response.notebook_utils import display_response


In [40]:
text_qa_template = PromptTemplate("""
You are a helpful AI tutor specialized in Anatomy and Physiology, designed to assist first-year MBBS students in understanding
complex concepts. You are provided with specific text from an Anatomy and Physiology textbook. Use only the provided context
to answer the student's question.

When referencing information from the provided context, cite the appropriate source(s) using their corresponding numbers.
Cite a source when you explicitly reference it. If none of the sources are helpful, indicate that. If multiple sources
contain the same information, use the source with the lower number. Answer in a polite, clear, concise, and educational manner,
focusing on conveying the most important points.

Important Guidelines:
Highlight key concepts: Emphasize the most important words or phrases in your answer.
Use bullet points: For clarity and emphasis, use bullet points and short headers where applicable.
Maintain a polite tone: Be respectful and supportive in your responses.

Probing for Clarification:
If the similarity between the query and the context is low, ask follow-up questions to gain clarity. Present these as bullet points.
If the student’s query is not related to Anatomy or Physiology, explain that you can only answer questions on these subjects.
Suggest a few relevant questions based on the context.

NOTE: - MAKE SURE THE RESPONSE IN CONCISE TO THE POINT
      - DONT USE THE WORD CONTEXT INSTEAD MENTION IT AS CONTENTS FROM THE ANATOMY AND PHYSIOLOGY TEXT BOOK

Context:
{context_str}

Student's Query:
{query_str}

Answer:
""")


In [100]:

query_gen_prompt_str = (
"""Generate search queries related to the input query: '{query}'.
 Ensure that each query is slightly refined or expanded by adding relevant context or closely related information,
 while maintaining the original intent and meaning of the query.
  NOTE : DO NOT EXPLAIN ANYTHING JUST RETURN THE QUERIES
 """
)

retriever = QueryFusionRetriever(
    [vector_retriever],
    similarity_top_k=2,
    num_queries=4,
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=query_gen_prompt_str,

)

query_engine = RetrieverQueryEngine.from_args(retriever,    text_qa_template = text_qa_template)

In [101]:
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.schema import ImageNode
import os
import requests
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

MAX_TOKENS = 50
retriever_engine = image_index.as_retriever(
    similarity_top_k=3, image_similarity_top_k=3
)

def retrieve_and_display(query):
    retrieval_results = retriever_engine.retrieve(query[:MAX_TOKENS])

    retrieved_images = []
    for res_node in retrieval_results:
        score = res_node.score

        if isinstance(res_node.node, ImageNode) :
            if score > 0.32:
              retrieved_images.append(res_node.node.metadata["file_path"])

    if retrieved_images:
        plt.figure(figsize=(15, 5))
        for i, img_path in enumerate(retrieved_images):
            plt.subplot(1, len(retrieved_images), i+1)
            img = Image.open(img_path)
            plt.imshow(img)
            plt.axis('off')
        plt.show()

In [102]:
def query_and_display_response(query_text: str):
    """
    Queries the engine with the provided query text, displays the response,
    retrieves additional information, and displays it.

    Parameters:
    query_text (str): The query string to be sent to the query engine.

    """
    # Query the engine and display the response
    response = query_engine.query(query_text)
    display_response(response)

    # Retrieve and display additional information based on the query text
    retrieve_and_display(query_text)


In [105]:
query_and_display_response("Explain me about newtons law of motion")


Generated queries:
1. What are the three laws of motion by Newton?
2. Explain Newton's first law of motion with examples
3. How does Newton's second law of motion relate force, mass, and acceleration?
4. What is the mathematical formula for Newton's second law of motion?
5. Describe Newton's third law of motion and provide real-life examples
6. How do Newton's laws of motion apply to everyday situations?
7. What is the difference between static and kinetic friction in relation to Newton's laws of motion?
8. How do Newton's laws of motion relate to the concept of inertia?
9. Explain the significance of Newton's laws of motion in the field of physics
10. What are some common misconceptions about Newton's laws of motion?


**`Final Response:`** I apologize, but Newton's laws of motion are not covered in the provided contents from the Anatomy and Physiology textbook. The textbook excerpts discuss Wolff's law related to bone remodeling and a question about Dalton's law and Henry's law.

If you have any questions related to the topics covered in the provided textbook contents, such as bone structure, modeling, remodeling, or gas laws, I would be happy to assist you. Here are a few relevant questions based on the given context:

- What is Wolff's law and how does it relate to bone remodeling?
- How would Wolff's law predict bone thickness in a right-handed pitcher's arms?
- What are the key differences between Dalton's law and Henry's law?

Please let me know if you have any other questions related to Anatomy and Physiology.